<a href="https://colab.research.google.com/github/sangjinsu/personalized-recommendation-system/blob/main/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

어떤 아이템에 대해 비슷한 취향을 가진 사람들은 다른 아이템에 대해서도 비슷한 취향을 가질 것이다.


### 유사도 지표 
 
- 상관계수  -1 ~ 1 사이 값 
- 코사인 유사도 
  - 협업 필터링에서 가장 널리 쓰이는 유사도 
  - 각 아이템 => 하나의 차원, 사용자의 평가값 => 좌표값
  - 두 사용자의 평가값 유사 => theta는 작아지고 코사인은 커진다 
  - -1 ~ 1 사이의 값
  - 데이터 이진값 => 타니모토 계수 사용 권장 
- 자카드 계수 
  - 타니모토 계수의 변형 => 자카드 계수
  - 이진수 데이터 => 좋은 결과 

### 기본 CF 알고리즘
1. 모든 사용자 간 평가의 유사도 계산
2. 추천 대상과 다른 사용자간 유사도 추출
3. 추천 대상이 평가하지 않은 아이템에 대해 예상 평가값 계산
4. 아이템 중 예상 평가값이 가장 높은 N개 추천 


In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = ''
u_user_src = os.path.join(base_src, 'u.user')

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date', 'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src, 
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# RMSE 계산함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 모델별 RMSE 계산 함수
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 코사인 유사도 

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


### 주어진 영화의 movie_id 가중 평균 rating을 계산하는 함수 
def CF_simple(user_id, movie_id):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating 

score(CF_simple)



1.0191514588846329

### 이웃을 고려한 CF

단순 CF 알고리즘 개선 방법
1. K Nearest Neighbors 방법
2. Thresholding 방법



In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = ''
u_user_src = os.path.join(base_src, 'u.user')

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date', 'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src, 
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# RMSE 계산함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 유사집단 크기를 미리 정하기 위해서 기존 score 함수에 neighnbor_size 인자값 추가 
def score(model, neighbor_size=0):
  # 테스트 데이터의 user_id 와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성 
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  # 실제 평점값 
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 코사인 유사도를 계산하는 사이킷런 라이브러리
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도를 구하기 위해 rating 값을 복사하고 계산 시 NaN 값 에러 대비를 위해 결측치 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정 
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


### KNN 정해서 예측치를 계산하는 함수 
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_socres = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_socres[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0
  return mean_rating 

score(CF_knn, neighbor_size=30)




1.0099391859451443

In [7]:
### 실제 주어진 사용자에 대해 추천을 받는 기능 구현
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
# 코사인 유사도를 구하기 위해 rating 값을 복사하고 계산 시 NaN 값 에러 대비를 위해 결측치 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정 
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)

def recom_movie(user_id, n_items, neighbor_size = 30):
  user_movie = rating_matrix.loc[user_id].copy()
  for movie in rating_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0

    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)
  
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(user_id=1, n_items=10, neighbor_size=30)

movie_id
1189                Prefontaine (1997)
1463                  Boys, Les (1997)
1656                Little City (1998)
1367                      Faust (1994)
12          Usual Suspects, The (1995)
1612           Leading Man, The (1996)
174     Raiders of the Lost Ark (1981)
408              Close Shave, A (1995)
1594                    Everest (1998)
1449            Pather Panchali (1955)
Name: title, dtype: object

### 최적의 이웃 크기 결정



In [8]:
for neighbor_size in range(10, 70, 10):
  print(score(CF_knn, neighbor_size))

1.0296971780308377
1.0120758679384685
1.0099391859451443
1.0101633158556151
1.010549526274457
1.0113106128194362


### 사용자 평가 경향을 고려한 CF 

1. 각 사용자 평점평균 계산
2. 평점 => 각 사용자의 평균에서의 차이로 변환
  - (평점 - 해당 사용자의 평균 평점)
3. 평점 편차의 예측값 계산
  - 평가값 = 평점편차  * 다른 사용자 유사도 
4. 실제 예측값 = 평점편차 예측값 + 평점 평균 


In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
### 코사인 유사도를 계산하는 사이킷런 라이브러리
from sklearn.metrics.pairwise import cosine_similarity

base_src = ''
u_user_src = os.path.join(base_src, 'u.user')

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date', 'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src, 
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# RMSE 계산함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 유사집단 크기를 미리 정하기 위해서 기존 score 함수에 neighnbor_size 인자값 추가 
def score(model, neighbor_size=0):
  # 테스트 데이터의 user_id 와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성 
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  # 실제 평점값 
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# 코사인 유사도를 구하기 위해 rating 값을 복사하고 계산 시 NaN 값 에러 대비를 위해 결측치 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정 
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


### 사용자 평가 경향을 고려한 함수 
rating_mean = ratings_matrix.mean(axis=1)
rating_bias = (ratings_matrix.T - rating_mean).T

### 사용자 평가 경향을 고려한 함수 
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
  if movie_id in rating_bias.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]
    
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]
      else:
        prediction = rating_mean[user_id]
  else:
    prediction = rating_mean[user_id] 
  return prediction 

score(CF_knn_bias, 30)

0.9344024862487913

### 그 외의 CF 정확도 개선 방법



In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
### 코사인 유사도를 계산하는 사이킷런 라이브러리
from sklearn.metrics.pairwise import cosine_similarity

base_src = 'drive/MyDrive/recommend'
u_user_src = os.path.join(base_src, 'u.user')

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date', 'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src, 
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# RMSE 계산함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 유사집단 크기를 미리 정하기 위해서 기존 score 함수에 neighnbor_size 인자값 추가 
def score(model, neighbor_size=0):
  # 테스트 데이터의 user_id 와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성 
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  # 실제 평점값 
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# 코사인 유사도를 구하기 위해 rating 값을 복사하고 계산 시 NaN 값 에러 대비를 위해 결측치 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정 
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


### 사용자 평가 경향을 고려한 함수 
rating_mean = ratings_matrix.mean(axis=1)
rating_bias = (ratings_matrix.T - rating_mean).T

####################################################################
rating_binary_1 = np.array(ratings_matrix>0).astype(float)
rating_binary_2 = rating_binary_1.T

counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts, 
                      index=ratings_matrix.index,
                      columns=ratings_matrix.index).fillna(0)

def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias.columns:
      sim_scores = user_similarity[user_id].copy()
      movie_ratings = rating_bias[movie_id].copy()

      no_rating = movie_ratings.isnull()
      common_counts = counts[user_id]
      low_significance = common_counts < SIG_LEVEL

      none_rating_idx = movie_ratings[no_rating | low_significance].index
      movie_ratings = movie_ratings.drop(none_rating_idx)
      sim_scores = sim_scores.drop(none_rating_idx)

      if neighbor_size == 0:
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]
    
      else:
        if len(sim_scores) > MIN_RATINGS:
          neighbor_size = min(neighbor_size, len(sim_scores))
          sim_scores = np.array(sim_scores)
          movie_ratings = np.array(movie_ratings)
          user_idx = np.argsort(sim_scores)
          sim_scores = sim_scores[user_idx][-neighbor_size:]
          movie_ratings = movie_ratings[user_idx][-neighbor_size:]
          prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
          prediction = prediction + rating_mean[user_id]
        else:
          prediction = rating_mean[user_id]
    else:
      prediction = rating_mean[user_id] 

    if prediction <= 1:
      prediction = 1
    elif prediction >= 5:
      prediction = 5

    return prediction 

SIG_LEVEL = 3
MIN_RATINGS = 10
score(CF_knn_bias_sig, 30)

0.9424209013514165

### 사용자 기반 CF와 아이템 기반 CF

- UBCF
  - 사용자 기반 CF
  - 데이터가 풍부한 경우 정확한 추천 
  - 결과에 대한 위험성 존재
- IBCF
  - 계산이 빠름
  - 업데이트에 대한 결과 영향이 적음
  - 아마존, 넷플릭스

유사도 계산 기준이 사용자인가 아이템인가에 따라 다름

데이터 크기가 적고 사용자에 대한 정보가 있는 경우 사용자 기반 CF 가 적절하다 

데이터 크기가 크고 충분한 정보가 없는 경우 아이템 기반 CF가 적절하다


In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
### 코사인 유사도를 계산하는 사이킷런 라이브러리
from sklearn.metrics.pairwise import cosine_similarity

base_src = ''
u_user_src = os.path.join(base_src, 'u.user')

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date', 'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src, 
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# RMSE 계산함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 유사집단 크기를 미리 정하기 위해서 기존 score 함수에 neighnbor_size 인자값 추가 
def score(model):
  # 테스트 데이터의 user_id 와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성 
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  # 실제 평점값 
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


####################################
rating_matrix_t = np.transpose(rating_matrix)

matrix_dummy = rating_matrix_t.copy().fillna(0)

item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity,
                               index=rating_matrix_t.index,
                               columns=rating_matrix_t.index)

def CF_IBCF(user_id, movie_id):
  if movie_id in item_similarity.columns:
    sim_scores = item_similarity[movie_id]
    user_rating = rating_matrix_t[user_id]
    none_rating_idx = user_rating[user_rating.isnull()].index
    user_rating = user_rating.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

score(CF_IBCF)

1.0163003637579802

### 추천 시스템의 성과측정지표

1. 데이터를 train_set 과 test_set 으로 분리 
2. train_set을 사용해서 학습하고 test set으로 평가 
3. 예상 평점과 실제 평점 차이를 계산 후 정확도 측정 

#### 성과 측정 지표
1. 각 아이템의 예상 평점과 실제 평점 차이
  - RMSE
2. 추천한 아이템과 사용자 실제 선택과 비교
  - 정확도
  - 정밀도 
  - 재현율
  - 정밀도와 재현율의 조화 평균
  - 범위 
  - 정밀도와 재형율은 trade-off 관계 